In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nd = pd.read_csv('../mtsinai/data/extra/numeric_data.csv')
#print(nd)

output = ["stais1","stais3","stais4","stais5","stais6","stais7","stais8","stais9","stais10","stais11","stais12","stais13",
          "stais14","stais15","stais16","stais17","stais18","stais19","stais20",
          "sds_1","sds_3","sds_4","stait5","sds_6","sds_7","sds_8","sds_9","sds_10","sds_11","sds_12","sds_13",
          "sds_14","sds_15","sds_16","sds_17","sds_18","sds_19","sds_20"]

data_y = nd.loc[:,output]
data_x = nd

columns = nd.columns.values

for col in columns:
    if col in output:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif nd[col].isna().sum() != 0:
        data_x = data_x.drop([col],axis = 1)
        #print(col)
    elif nd[col].dtypes == object:
        data_x = data_x.drop([col],axis = 1)
        #print(nd[col].dtypes)

    '''
    elif not nd[col].dtypes is np.dtype('int64'):
        if not nd[col].dtypes is np.dtype('float64'):
            data_x = data_x.drop([col],axis = 1)
            print(nd[col].dtypes)
    ''' 

#data_y = nd.loc[:,["stai_s_score"]]

data_x = np.array(data_x, dtype=np.float32)
data_y = np.array(data_y, dtype=np.float32)


X = data_x[:1316]
y = data_y[:1316]
X_test = data_x[1316:]
y_test = data_y[1316:]

print(data_x.shape)
print(data_y.shape)
#print(data_t.shape)

print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)


(1463, 287)
(1463, 38)
(1316, 287)
(1316, 38)
(147, 287)
(147, 38)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


x1 = np.delete(X,slice(263),0)
x2 = np.delete(X,slice(263,526),0)
x3 = np.delete(X,slice(526,790),0)
x4 = np.delete(X,slice(790,1053),0)
x5 = X[:1053]


y1 = np.delete(y,slice(263),0)
y2 = np.delete(y,slice(263,526),0)
y3 = np.delete(y,slice(526,790),0)
y4 = np.delete(y,slice(790,1053),0)
y5 = y[:1053]

X_groups = [x1,x2,x3,x4,x5]
y_groups = [y1,y2,y3,y4,y5]
X_vali = [X[:263],X[263:526],X[526:790],X[790:1053],X[1053:]]
y_vali = [y[:263],y[263:526],y[526:790],y[790:1053],y[1053:]]


class TwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x


input_size = X.shape[1]
hidden_size = 100
output_size = y.shape[1]

learning_rate = 0.01
num_epochs = 100


# Training
mse = 0
for i in range(5):
    X_tensor = torch.tensor(X_groups[i])
    y_tensor = torch.tensor(y_groups[i])
    
    X_validation = torch.tensor(X_vali[i])
    y_validation = torch.tensor(y_vali[i])
    
    
    model = TwoLayerNN(input_size, hidden_size, output_size)

    
    criterion = nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    

    for epoch in range(num_epochs):
        
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 20 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")
    
    predictions = model(X_validation)
    loss = criterion(predictions, y_validation)
    print("Validation loss:",str(loss.item()))
    mse += loss.item()

CV_k = mse / 5
print(CV_k)

Epoch 0, Loss: 17.38205909729004
Epoch 20, Loss: 0.8897262215614319
Epoch 40, Loss: 0.7352638244628906
Epoch 60, Loss: 0.5524552464485168
Epoch 80, Loss: 0.5232217311859131
Validation loss: 0.5096479058265686
Epoch 0, Loss: 20.955528259277344
Epoch 20, Loss: 0.9421632885932922
Epoch 40, Loss: 0.6932333111763
Epoch 60, Loss: 0.5396820902824402
Epoch 80, Loss: 0.5108973979949951
Validation loss: 0.5424189567565918
Epoch 0, Loss: 15.72448444366455
Epoch 20, Loss: 0.9888092279434204
Epoch 40, Loss: 0.7991477251052856
Epoch 60, Loss: 0.6627576351165771
Epoch 80, Loss: 0.5367245674133301
Validation loss: 0.5384560823440552
Epoch 0, Loss: 27.38916778564453
Epoch 20, Loss: 0.9578471779823303
Epoch 40, Loss: 0.843695342540741
Epoch 60, Loss: 0.799744725227356
Epoch 80, Loss: 0.7732215523719788
Validation loss: 0.7193886637687683
Epoch 0, Loss: 27.157001495361328
Epoch 20, Loss: 0.9057052731513977
Epoch 40, Loss: 0.773781418800354
Epoch 60, Loss: 0.5997708439826965
Epoch 80, Loss: 0.555132448673

In [3]:
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)


model = TwoLayerNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)


with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)

#print(predictions)
print(f"Test Loss: {test_loss.item()}")


Epoch 0, Loss: 16.480609893798828
Epoch 20, Loss: 0.9433692097663879
Epoch 40, Loss: 0.7890931367874146
Epoch 60, Loss: 0.6122169494628906
Epoch 80, Loss: 0.5411585569381714
Test Loss: 0.5636346936225891


In [4]:
def scores(data):
    return [sum(((5-data[0]),(data[1]),(data[2]),(5-data[3]),(data[4]),(data[5]),(5-data[6]),(data[7]),
               (5-data[8]),(5-data[9]),(data[10]),(data[11]),(data[12]),(5-data[13]),(5-data[14]),
               (data[15]),(data[16]),(5-data[17]),(5-data[18]))),
            sum(((data[19]),(data[20]),(data[21]),(5-data[22]),(5-data[23]),(data[24]),(data[25]),(data[26]),
               (data[27]),(5-data[28]),(5-data[29]),(data[30]),(5-data[31]),(data[32]),(5-data[33]),
               (5-data[34]),(5-data[35]),(data[36]),(5-data[37])))
           ]


#print([int(i+0.5) for i in predictions[0]])
#print(y_test[0].shape)

predict = predictions.tolist()
print(predict)

p_scores = [scores(i) for i in predict]
y_scores = [scores(i) for i in y_test]
#print(p_scores)

p_sc = [[int(i[0]+0.5),int(i[1]+0.5)] for i in p_scores]
y_sc = [[int(j[0]),int(j[1])] for j in y_scores]
print(p_sc)
print()
print(y_sc)


p_s = torch.tensor(p_scores)
y_s = torch.tensor(y_scores)


l = criterion(p_s, y_s)
print("\nMSEloss:")
print(l.item())

[[2.081451892852783, 2.444596290588379, 2.3962559700012207, 1.7475378513336182, 1.8382465839385986, 2.7600481510162354, 1.610745906829834, 2.070673942565918, 1.9383454322814941, 1.569688081741333, 2.337047815322876, 2.031132698059082, 2.1445844173431396, 1.6850858926773071, 1.634986162185669, 2.5653600692749023, 1.8431310653686523, 1.7049927711486816, 1.6644725799560547, 2.2957518100738525, 1.8791412115097046, 2.62127947807312, 2.5387258529663086, 2.217904806137085, 1.5741753578186035, 1.7466802597045898, 1.9108806848526, 2.520662307739258, 1.7819693088531494, 1.6859923601150513, 2.078230619430542, 1.8750336170196533, 2.386843681335449, 1.6856701374053955, 1.8294178247451782, 1.7968870401382446, 1.7526638507843018, 2.006943464279175], [2.770615816116333, 1.6354385614395142, 1.4893767833709717, 2.4821012020111084, 1.2472039461135864, 1.7382357120513916, 2.3965084552764893, 1.291588544845581, 2.528214454650879, 2.4188756942749023, 1.5572344064712524, 1.3476728200912476, 1.340960264205932

In [ ]:
#lime
from lime import lime_tabular



In [7]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score, recall_score, f1_score

# yp = np.array(predictions)
# yt = y_test.astype(int)
# a = [int(i+0.5) for i in yp[:,0]]
# print(a)
# print(yt[:,0])

# for i in range(yp.shape[1]):
#     ypi = yp[:,i]
#     yti = np.array(yt[:,i])
#     #print(ypi)
#     ypi = np.array([int(i+0.5) for i in ypi])
    
#     acc = accuracy_score(yti, ypi)
#     print(acc)
#     print()
#     print(confusion_matrix(yti, ypi))
#     print()
    
#     p = precision_score(yti, ypi, average='micro')
#     p2 = precision_score(yti, ypi, average='macro')
#     p3 = precision_score(yti, ypi, average='weighted')
    
#     r = recall_score(yti, ypi, average='micro')
#     r2 = recall_score(yti, ypi, average='macro')
#     r3 = recall_score(yti, ypi, average='weighted')
    
#     fs = f1_score(yti, ypi, average='micro')
#     fs2 = f1_score(yti, ypi, average='macro')
#     fs3 = f1_score(yti, ypi, average='weighted')
    
#     print(p,p2,p3)
#     print(r,r2,r3)
#     print(fs,fs2,fs3)
#     print("\n\n")
    

In [8]:
# from sklearn.metrics import precision_recall_curve, average_precision_score
# import numpy as np


# def Auprc(y_t, y_s):
#     y_true = y_t
#     y_scores = y_s

#     precision = dict()
#     recall = dict()
#     average_precision = dict()
#     ap = []

#     for i in range(np.max(y_true) + 1):
#         y_true_i = [1 if label == i else 0 for label in y_true]
#         precision[i], recall[i], _ = precision_recall_curve(y_true_i, y_scores)
#         average_precision[i] = average_precision_score(y_true_i, y_scores)
#         ap.append(average_precision_score(y_true_i, y_scores))

#     precision["micro"], recall["micro"], _ = precision_recall_curve(np.array([1 if label > 0 else 0 for label in y_true]), y_scores)
#     average_precision["micro"] = average_precision_score(np.array([1 if label > 0 else 0 for label in y_true]), y_scores)

#     #average_precision["macro"] = np.mean(list(average_precision.values()))
#     average_precision["macro"] = np.mean(ap)

#     for i in range(np.max(y_true) + 1):
#         print(f"Class {i} - AUPRC: {average_precision[i]}")

#     print(f"Micro-average AUPRC: {average_precision['micro']}")
#     print(f"Macro-average AUPRC: {average_precision['macro']}")

    


# y_t = y_test.astype(int)
# #print(y_t)

# y_s = np.array(predictions)

# for i in range(y_t.shape[1]):
#     yti = y_t[:,i]
#     ysi = y_s[:,i]
#     #ysi = [int(i+0.5) for i in ysi]
#     Auprc(yti, ysi)
#     print(i)
    

